In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("CovidAnalysis").getOrCreate()

covid_df = spark.read.csv("/home/jovyan/work/covid-data.csv", header=True, inferSchema=True)

In [4]:
#Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)
from pyspark.sql.functions import max, lit

march_31_df = covid_df.filter(col("date") == "2021-03-31")
march_31_df = march_31_df.withColumn("perc_recovered", (col("total_cases") / col("population")) * 100)
top_15_countries = march_31_df.select("iso_code", "location", "perc_recovered") \
                               .orderBy(col("perc_recovered").desc()) \
                               .limit(15)
top_15_countries.write.mode("overwrite").csv("/home/jovyan/top_15_countries.csv", header=True)

In [5]:
#Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
#(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)
from pyspark.sql import functions as F

# Фильтруем ненужные записи
valid_countries_df = covid_df.filter(~covid_df['location'].isin("World", "Asia", "Africa"))

# Находим максимальное количество случаев и дату, когда оно зафиксировано
top_10_countries = valid_countries_df \
    .filter((F.col('date') >= '2021-03-24') & (F.col('date') <= '2021-03-31')) \
    .groupBy("location") \
    .agg(F.max("new_cases").alias("max_new_cases"),
         F.first("date").alias("date_of_max_cases")) \
    .orderBy(F.desc("max_new_cases")) \
    .limit(10)

top_10_countries.show()
top_10_countries.write.mode("overwrite").csv("/home/jovyan/top_10_countries.csv", header=True)

+--------------+-------------+-----------------+
|      location|max_new_cases|date_of_max_cases|
+--------------+-------------+-----------------+
|        Europe|     256192.0|       2021-03-24|
|European Union|     216452.0|       2021-03-24|
| South America|     148476.0|       2021-03-24|
| North America|     100760.0|       2021-03-24|
|        Brazil|     100158.0|       2021-03-24|
| United States|      86960.0|       2021-03-24|
|         India|      72330.0|       2021-03-24|
|        France|      65392.0|       2021-03-24|
|        Turkey|      39302.0|       2021-03-24|
|        Poland|      35145.0|       2021-03-24|
+--------------+-------------+-----------------+



In [7]:
#Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. 
#(например: в россии вчера было 9150 , сегодня 8763, итог: -387) 
#(в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)
from pyspark.sql.window import Window

window_spec = Window.partitionBy("location").orderBy("date")

russia_df = covid_df.filter(covid_df['location'] == "Russia") \
    .withColumn("daily_new_cases", F.col("new_cases") - F.lag("new_cases", 1).over(window_spec)) \
    .na.fill(0, subset=["daily_new_cases"])

russia_df = russia_df.select("date", "daily_new_cases").filter(F.col("date") >= '2021-03-24')

russia_df.show()
russia_df.write.mode("overwrite").csv("/home/jovyan/russia_delta_cases.csv", header=True)

+----------+---------------+
|      date|daily_new_cases|
+----------+---------------+
|2021-03-24|          400.0|
|2021-03-25|          359.0|
|2021-03-26|          -55.0|
|2021-03-27|         -290.0|
|2021-03-28|          196.0|
|2021-03-29|         -390.0|
|2021-03-30|         -427.0|
|2021-03-31|           -6.0|
|2021-04-01|          901.0|
|2021-04-02|         -375.0|
|2021-04-03|          224.0|
|2021-04-04|         -209.0|
|2021-04-05|         -172.0|
|2021-04-06|         -319.0|
|2021-04-07|          -32.0|
|2021-04-08|          371.0|
|2021-04-09|          482.0|
|2021-04-10|         -447.0|
|2021-04-11|           -4.0|
|2021-04-12|         -385.0|
+----------+---------------+
only showing top 20 rows

